# __이미지로 이미지 검색하기__

**이미지 수치화 기술 이해하기**
---
👉 이미지는 고차원 데이터(높이x너비x채널(RGB)x색의 강도)로써 각 픽셀의 정보를 무작위로 생성한다면 아무 의미를 갖지 않는 이미지가 생성됩니다. 즉, 각 픽셀은 주변 픽셀과 연관된 특정 패턴을 갖는 경우에만 사람이 이미지로 인식을 할 수 있게 됩니다. 이는 이미지를 실제보다 저차원적 특성 벡터에 표현할 수 있다는 의미입니다. 최근에는 인공지능을 이용해서 각 이미지가 갖는 의미의 유사도에 따라 저차원 공간에 수치화하여 표현하는 연구들이 다방면으로 이루어지고 있으며 이는 이미지 수치화, 벡터화(Vectorization), 임베딩(Embedding) 등의 이름으로 다양하게 활용되고 있습니다.

**본 튜토리얼에서는**
---
👉 이번 튜토리얼에서는 MNIST 손글씨 데이터 세트를 사용합니다. 각 이미지는 0-1 사이의 값을 갖는 고정 크기(28x28 = 784 픽셀) 이며, 여러 사람들이 손으로 쓴 0-9까지 숫자를 정답과 함께 제공합니다. 1,000 개의 학습용 데이터 세트와 200개의 테스트용 데이터 세트로 이루어져 있습니다.

ThanoSQL을 사용하여 손글씨 데이터를 입력하고 DB 내에서 입력 이미지와 유사한 이미지를 검색해주는 모델을 만들어 봅니다.

## __0. 데이터 세트 확인__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/quick_start/how_to_use_ThanoSQL/#5-thanosql)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은 개인 토큰>

In [5]:
%%thanosql
COPY mnist_train 
OPTIONS(overwrite = True)
FROM "tutorial_data/mnist_data/mnist_train.csv"

Time Taken: 0.51 seconds
Success


In [7]:
%%thanosql
COPY mnist_test 
OPTIONS(overwrite = True)
FROM "tutorial_data/mnist_data/mnist_test.csv"

Time Taken: 0.45 seconds
Success


__OPTIONS__ : 

__overwrite가 True일 때__, 사용자는 이전 생성했던 데이터 테이블과 같은 이름의 데이터 테이블을 생성할 수 있습니다.  
반면, __overwrite가 False일 때__, 사용자는 이전에 생성했던 데이터 테이블과 같은 이름의 데이터 테이블을 생성할 수 없습니다.

## __1. 데이터 세트 확인__

손글씨 분류 모델을 만들기 위해 ThanoSQL DB에 저장되어 있는 mnist_train 테이블을 사용합니다. mnist_train 테이블은 MNIST 이미지 파일들이 저장되어 있는 경로와 파일 이름 그리고 라벨 정보가 담겨 있는 테이블입니다. 아래의 쿼리문을 실행하고 테이블의 내용을 확인합니다.

In [8]:
%%thanosql
SELECT * 
FROM mnist_train 
LIMIT 5

Time Taken: 0.44 seconds


,image_path,filename,label
0,tutorial_data/mnist_data/train/6782.jpg,6782.jpg,5
1,tutorial_data/mnist_data/train/1810.jpg,1810.jpg,5
2,tutorial_data/mnist_data/train/33617.jpg,33617.jpg,5
3,tutorial_data/mnist_data/train/27802.jpg,27802.jpg,5
4,tutorial_data/mnist_data/train/50677.jpg,50677.jpg,5


mnist_train 테이블은 아래와 같은 정보를 담고 있습니다. "6782.jpg" 이미지 파일은 숫자 5를 쓴 손글씨 이미지입니다.

- img_path: 이미지 경로
- filename: 파일 이름
- label : 이미지 라벨

## __2. 이미지 수치화 모델 생성__

이전 단계에서 확인한 mnist_train 테이블을 사용하여 이미지 수치화 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 my_image_search_model이라는 이름의 모델을 만듭니다.  

(쿼리 실행 시 예상 소요 시간 : 1 min)

In [30]:
%%thanosql
BUILD MODEL my_image_search_model
USING SimCLR
OPTIONS (
    image_col="img_path",
    max_epochs=5,
    overwrite=True
    )
AS 
SELECT * 
FROM mnist_train

Time Taken: 332.16 seconds
Success


# __쿼리 세부 정보__ 
- "__BUILD MODEL__" 쿼리 구문을 사용하여 my_image_search_model 이라는 모델을 만들고 학습시킵니다.
- "__USING__" 쿼리 구문을 통해 베이스 모델로 SimCLR 모델을 사용할 것을 명시합니다.
- "__OPTIONS__" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.  
    -  "image_col" : 데이터 테이블에서 이미지의 경로를 담은 컬럼 (Default : "image_path")    
    -  "max_epochs" : 이미지 수치화 모델을 생성하기 위한 데이터 세트 학습 횟수

아래 쿼리 구문을 사용하여 이미지 수치화 결과를 확인합니다. `my_image_search_model`을 "__CONVERT USING__" 쿼리 구문을 사용하여 `mnist_test` 이미지들을 임베딩합니다. 

In [37]:
%%thanosql
CONVERT USING my_image_search_model
OPTIONS (
    table_name= "mnist_test",
    image_col="img_path"
    )
AS 
SELECT * 
FROM mnist_test

Time Taken: 2.19 seconds


,image_path,filename,label,my_image_search_model_simclr
0,tutorial_data/mnist_data/test/5099.jpg,5099.jpg,6,"[0.8018283247947693, 0.79114830493927, 1.35917..."
1,tutorial_data/mnist_data/test/9239.jpg,9239.jpg,6,"[0.5109610557556152, 0.7103637456893921, 0.762..."
2,tutorial_data/mnist_data/test/2242.jpg,2242.jpg,6,"[0.5548616051673889, 0.5135034322738647, 0.882..."
3,tutorial_data/mnist_data/test/3451.jpg,3451.jpg,6,"[0.6456339955329895, 0.8264403939247131, 0.982..."
4,tutorial_data/mnist_data/test/2631.jpg,2631.jpg,6,"[0.9043697118759155, 1.001702070236206, 0.8401..."
...,...,...,...,...
195,tutorial_data/mnist_data/test/8045.jpg,8045.jpg,8,"[0.900598406791687, 0.8024667501449585, 0.9821..."
196,tutorial_data/mnist_data/test/9591.jpg,9591.jpg,8,"[0.7340981960296631, 0.24722115695476532, 1.25..."
197,tutorial_data/mnist_data/test/7425.jpg,7425.jpg,8,"[0.6172817349433899, 0.278918594121933, 0.4667..."
198,tutorial_data/mnist_data/test/2150.jpg,2150.jpg,8,"[1.0326189994812012, 0.8284595012664795, 0.923..."


__쿼리 세부 정보__ 
- "__CONVERT USING__" 쿼리 구문은 `my_image_search_model`을 이미지 수치화를 위한 알고리즘으로 사용합니다.   
- "__OPTIONS__" 쿼리 구문을 통해 이미지 수치화 시 필요한 변수들을 정의합니다. 
    - "table_name" : ThanoSQL DB 내에 저장될 테이블 이름을 정의합니다.
    - "image_col" : 데이터 테이블에 이미지 파일 경로가 담겨있는 칼럼명을 정의합니다. (DEFAULT: "image_path")

`mnist_test` 테이블에 `my_image_search_model_simclr`이라는 컬럼을 새롭게 생성하고 수치화 결과를 저장합니다.

## __3. 이미지 수치화 모델을 사용해서 유사 이미지 검색하기__

이번 단계에서는 my_image_search_model 이미지 수치화 모델과 테스트 테이블을 사용하여 "923.jpg" 이미지 파일(손글씨 8)과 유사한 이미지를 검색합니다.

In [41]:
%%thanosql
SEARCH IMAGE images='tutorial_data/mnist_data/test/923.jpg' 
USING my_image_search_model 
AS
SELECT * 
FROM mnist_test

Time Taken: 6.62 seconds


,image_path,my_image_search_model_simclr,my_image_search_model_simclr_similarity1
0,tutorial_data/mnist_data/test/2778.jpg,"[0.4345078468322754, 0.4801121950149536, 0.849...",0.904738
1,tutorial_data/mnist_data/test/616.jpg,"[0.8675670623779297, 1.101262092590332, 0.7946...",0.940482
2,tutorial_data/mnist_data/test/1809.jpg,"[0.911918044090271, 0.5684484839439392, 0.7830...",0.929987
3,tutorial_data/mnist_data/test/5165.jpg,"[1.4594722986221313, 1.5304396152496338, 0.906...",0.946843
4,tutorial_data/mnist_data/test/8600.jpg,"[0.7580893635749817, 0.579352617263794, 0.8556...",0.901757
...,...,...,...
195,tutorial_data/mnist_data/test/4949.jpg,"[0.8616079688072205, 1.2864174842834473, 0.899...",0.929408
196,tutorial_data/mnist_data/test/6146.jpg,"[0.5501160621643066, 0.8333878517150879, 1.228...",0.933200
197,tutorial_data/mnist_data/test/2002.jpg,"[1.0146446228027344, 0.6418188810348511, 0.629...",0.933350
198,tutorial_data/mnist_data/test/4255.jpg,"[0.8223161697387695, 0.5770737528800964, 1.053...",0.936178


__쿼리 세부 정보__  

- "__SEARCH IMAGE images=__" 쿼리 구문은 검색하고자 하는 이미지 파일을 정의합니다.  <br>
- "__USING__"은 이미지 수치화에 사용할 모델을 정의합니다.<br>
- "__AS__" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. `mnist_embds` 테이블을 사용합니다 

다음 쿼리를 실행하여 "__SEARCH__" 결과를 ThanoSQL의 "__PRINT__" 쿼리 구문을 활용하여 가장 유사한 상위 4개를 출력합니다. 학습을 조금 밖에 진행하지 않았지만 8과 비슷한 이미지를 출력하는 것을 확인할 수 있습니다.

In [42]:
%%thanosql
PRINT IMAGE 
AS (
    SELECT image_path, my_image_search_model_simclr_similarity1 
    FROM (
        SEARCH IMAGE images='tutorial_data/mnist_data/test/923.jpg' 
        USING my_image_search_model 
        AS 
        SELECT * 
        FROM mnist_test
        )
    ORDER BY my_image_search_model_simclr_similarity1 DESC 
    LIMIT 4
    )

Time Taken: 1.12 seconds
/home/jovyan/tutorial_data/mnist_data/test/923.jpg


/home/jovyan/tutorial_data/mnist_data/test/685.jpg


/home/jovyan/tutorial_data/mnist_data/test/6573.jpg


/home/jovyan/tutorial_data/mnist_data/test/5087.jpg


__참고 사항__ 

이미지 유사도 검색 알고리즘의 기본 학습 옵션은 이미지의 좌우상하 반전, 색상의 변화 등에 관계없이 모두 같은 이미지로 인식하도록 학습이 진행 됩니다. 강아지의 사진은 뒤집히거나 색이 변해도 강아지로 인식되어야 하기 때문입니다. 의류 이미지 등과 같이 색의 변화가 중요하거나 숫자 처럼 상하, 좌우 반전이 중요한 경우 학습 시 옵션을 변경해 주어야 합니다. 본 튜토리얼에서는 이러한 이미지 유사도 검색의 특징을 보여주고 있습니다.